This ground truths from bags onto a map to visualize if there are any overlapping sequences.

In [2]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from typing import List, Tuple
import pandas as pd
import rosbag
from os import path
import numpy as np


# GPS_TOPIC = "/novatel/oem7/inspva"
GPS_TOPIC = "/kitti/oxts/gps/fix"


def read_rosbag_gps(bag_path: str, gps_topic: str) -> pd.DataFrame:
    points = []
    print(f"reading {bag_path}")
    with rosbag.Bag(bag_path, "r") as bag:
        messages = bag.read_messages(topics=[gps_topic])
        for topic, msg, timestamp in messages:
            # points.append((msg.latitude, msg.longitude, msg.height))
            points.append((msg.latitude, msg.longitude, msg.altitude))

    return pd.DataFrame(points, columns=["lat", "lon", "alt"])


# data = {
#     "seq_01": read_rosbag_gps(path.expanduser("~/bags/seq_01.bag"), GPS_TOPIC),
#     "seq_02": read_rosbag_gps(path.expanduser("~/bags/seq_02.bag"), GPS_TOPIC),
#     "seq_03": read_rosbag_gps(path.expanduser("~/bags/seq_03.bag"), GPS_TOPIC),
# }


data = {
    "kitti_2011_09_30_drive_0016": read_rosbag_gps(path.expanduser("~/shared-data/2011_09_30/kitti_2011_09_30_drive_0016_synced_.bag"), GPS_TOPIC),
    "kitti_2011_09_30_drive_0018": read_rosbag_gps(path.expanduser("~/shared-data/2011_09_30/kitti_2011_09_30_drive_0018_synced_.bag"), GPS_TOPIC),
    "kitti_2011_09_30_drive_0020": read_rosbag_gps(path.expanduser("~/shared-data/2011_09_30/kitti_2011_09_30_drive_0020_synced_.bag"), GPS_TOPIC),
    "kitti_2011_09_30_drive_0027": read_rosbag_gps(path.expanduser("~/shared-data/2011_09_30/kitti_2011_09_30_drive_0027_synced_.bag"), GPS_TOPIC),
    "kitti_2011_10_03_drive_0027": read_rosbag_gps(path.expanduser("~/shared-data/2011_10_03/kitti_2011_10_03_drive_0027_synced_.bag"), GPS_TOPIC),
    "kitti_2011_10_03_drive_0034": read_rosbag_gps(path.expanduser("~/shared-data/2011_10_03/kitti_2011_10_03_drive_0034_synced_.bag"), GPS_TOPIC),
    "kitti_2011_10_03_drive_0042": read_rosbag_gps(path.expanduser("~/shared-data/2011_10_03/kitti_2011_10_03_drive_0042_synced_.bag"), GPS_TOPIC),
}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
reading /Users/erik/shared-data/2011_09_30/kitti_2011_09_30_drive_0016_synced_.bag
reading /Users/erik/shared-data/2011_09_30/kitti_2011_09_30_drive_0018_synced_.bag
reading /Users/erik/shared-data/2011_09_30/kitti_2011_09_30_drive_0020_synced_.bag
reading /Users/erik/shared-data/2011_09_30/kitti_2011_09_30_drive_0027_synced_.bag
reading /Users/erik/shared-data/2011_10_03/kitti_2011_10_03_drive_0027_synced_.bag
reading /Users/erik/shared-data/2011_10_03/kitti_2011_10_03_drive_0034_synced_.bag
reading /Users/erik/shared-data/2011_10_03/kitti_2011_10_03_drive_0042_synced_.bag


In [3]:
from gorbslam.common.plotting_utils import create_map_fig, create_scattermapbox


traces = []
for name, trajectory in data.items():
    traces.append(create_scattermapbox(trajectory, name, mode="lines", bold=True))

tmp = np.concatenate([*data.values()])
lat_center = np.mean(tmp[:, 0])
alt_center = np.mean(tmp[:, 1])

fig = create_map_fig(traces, (lat_center, alt_center))
fig.update_layout(
    height=1200, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)
fig.show()

In [4]:
# from typing import Dict
# import pandas as pd
# import geopandas as gpd
# import numpy as np
# from shapely.geometry import LineString
# import plotly.graph_objs as go


# def plotly_hex_to_rgba(color, alpha=1):
#     r, g, b = hex_to_rgb(color)
#     return f"rgba({r}, {g}, {b}, {alpha})"


# def plotly_average_colors(colors, alpha=1):
#     rgbs = [hex_to_rgb(color) for color in colors]
#     avg_rgb = [sum(c) / len(c) for c in zip(*rgbs)]
#     return plotly_hex_to_rgba(avg_rgb, alpha)


# def find_overlapping_sections(
#     data: Dict[str, pd.DataFrame], distance_threshold: float = 0.00005
# ) -> Dict[Tuple[str, str], pd.DataFrame]:
#     overlapping_sections = {}

#     names = list(data.keys())
#     for i in range(len(names)):
#         for j in range(i + 1, len(names)):
#             name1, name2 = names[i], names[j]
#             trajectory1, trajectory2 = data[name1], data[name2]

#             lat1, lon1 = trajectory1["lat"].values, trajectory1["lon"].values
#             lat2, lon2 = trajectory2["lat"].values, trajectory2["lon"].values

#             overlap_indices = []

#             for index, (lat, lon) in enumerate(zip(lat1, lon1)):
#                 distances = np.sqrt((lat2 - lat) ** 2 + (lon2 - lon) ** 2)
#                 close_points = np.where(distances <= distance_threshold)[0]

#                 if len(close_points) > 0:
#                     overlap_indices.append(index)

#             overlapping_sections[(name1, name2)] = trajectory1.loc[overlap_indices]

#     return overlapping_sections


# def plot_trajectories_with_overlapping_sections(data: Dict[str, pd.DataFrame]):
#     traces = []

#     for name, trajectory in data.items():
#         traces.append(
#             create_scattermapbox(trajectory, name, mode="lines", bold=True, opacity=0.5)
#         )

#     overlapping_sections = find_overlapping_sections(data)
#     for i, ((name1, name2), overlap) in enumerate(overlapping_sections.items()):
#         traces.append(
#             create_scattermapbox(
#                 overlap,
#                 f"Overlap {name1}-{name2}",
#                 # color=f"rgba(134,106,158,{(i + 1) * 0.2})",
#                 # color=f"rgba(134,106,158,0.3)",
#                 # color=f"rgb(134,106,158)",
#                 mode="lines",
#                 bold=True,
#                 opacity=0.6,
#             )
#         )

#     tmp = np.concatenate([*data.values()])
#     lat_center = np.mean(tmp[:, 0])
#     lon_center = np.mean(tmp[:, 1])

#     fig = create_map_fig(traces, (lat_center, lon_center))
#     fig.update_layout(
#         height=1200,
#         # legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
#     )
#     fig.show()


# plot_trajectories_with_overlapping_sections(data)